In [3]:
import os

In [4]:
%pwd

'd:\\Python_DataScience\\Wine_Quality_Prediction\\Wine_Quality_Prediction_End_To_End_ML\\research'

In [5]:
os.chdir("../")


In [6]:
%pwd

'd:\\Python_DataScience\\Wine_Quality_Prediction\\Wine_Quality_Prediction_End_To_End_ML'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [8]:
from wine_pred.constants import *

from wine_pred.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__ (
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config




In [25]:
from wine_pred.logging.logger import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [23]:
class DataTransformation:
    def __init__ (self, config: DataTransformationConfig):
        self.config = config

    def data_train_test_split(self):

        df = pd.read_csv(self.config.data_path)
        df = df.iloc[:,0:-1]
        train, test = train_test_split(df,test_size=0.25)

        columns_name = df.columns
        
        train_df = pd.DataFrame(data=train,columns=columns_name)
        test_df = pd.DataFrame(data=test,columns=columns_name)

        train_path = os.path.join(self.config.root_dir,"train.csv")
        test_path = os.path.join(self.config.root_dir,"test.csv")

        train.to_csv(train_path, index=False)
        test.to_csv(test_path, index=False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)
    

        
    

In [24]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.data_train_test_split()
    
except Exception as e:
    raise e

[ 2024-07-07 12:45:53,517 ] 29 root - INFO - yaml file: config\config.yaml loaded succesfully
[ 2024-07-07 12:45:53,519 ] 29 root - INFO - yaml file: params.yaml loaded succesfully
[ 2024-07-07 12:45:53,519 ] 29 root - INFO - yaml file: schema.yaml loaded succesfully
[ 2024-07-07 12:45:53,519 ] 43 root - INFO - Created directory at: artifacts
[ 2024-07-07 12:45:53,519 ] 43 root - INFO - Created directory at: artifacts/data_transformation
[ 2024-07-07 12:45:53,536 ] 22 root - INFO - Splited data into training and test sets
[ 2024-07-07 12:45:53,536 ] 23 root - INFO - (857, 12)
[ 2024-07-07 12:45:53,536 ] 24 root - INFO - (286, 12)


In [18]:
df = pd.read_csv('artifacts\data_ingestion\wine_quality_dataset\WineQT.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4
